# Meal Preparation Notebook
## Upload the data

Please upload the files named "Emission.csv" and
"Nutrient.csv" after running the chunk right below

In [ ]:
"""
Run this code cell to mount your Google Drive before running the following blocks
Upload the following files: "Emission.csv", "Nutrient.csv", which are included in the report
"""
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
files.upload()

Saving Emission.csv to Emission (1).csv
Saving Nutrient.csv to Nutrient.csv


{'Emission (1).csv': b'\xef\xbb\xbf,"Resampled, Randomized Data",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,"Data Without Resampling or Randomization (original study data, after harmonizing methodology)",,,,,,,,,,,\r\n,Land Use (m2/FU),,,,,,"GHG Emissions (kg CO2eq/FU, IPCC 2013 incl. CC feedbacks)",,,,,,"GHG Emissions (kg CO2eq/FU, IPCC 2007)",,,,,,"Acidifying Emissions (g SO2eq/FU, CML2 Baseline)",,,,,,"Eutrophying Emissions (g PO43-eq/FU, CML2 Baseline)",,,,,,Freshwater Withdrawals (L/FU),,,,,,Stress-Weighted Water Use (L/FU),,,,,,Land Use (m2/FU),,"GHG (kg CO2eq/FU, IPCC 2013 incl. CC feedbacks)",,"Acid. (kg SO2eq/FU, CML2 Baseline)",,"Eutr. (kg PO43-eq/FU, CML2 Baseline)",,Fresh W. (L/FU),,Strs. W. WU (L/FU),\r\nProduct,5th pctl,10th pctl,Mean,Median,90th pctl,95th pctl,5th pctl,10th pctl,Mean,Median,90th pctl,95th pctl,5th pctl,10th pctl,Mean,Median,90th pctl,95th pctl,5th pctl,10th pctl,Mean,Median,90th pctl,95th pctl,5th pctl,10th pctl,Mean,Median,90th pctl,95th pctl,5th pctl,10t

In [ ]:
# @title The Ingredient class and its assignment functions

class Ingredient:
    def __init__(self, ingred, unit, cal, pro, fat, car, GHG, ser = 0.008):
        self.ingredient = ingred
        self.unit = unit
        self.calorie_per_unit = cal
        self.serving_portion = ser
        self.emissions = GHG
        self.protein_per_unit = pro
        self.fat_per_unit = fat
        self.carb_per_unit = car

def append_to_list(carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list, i, L1, L2):
    nutrient_dict = {'CarbSource' : carb_list, 'FatSource': fat_list, 'ProteinSource': protein_list, 'Vegetable': vegetable_list, 'Fruit': fruit_list, 'Extra': extra_list}
    if L2[i][1] in nutrient_dict:
        nutrient_dict[L2[i][1]].append(Ingredient(L2[i][0], L2[i][2], L2[i][4], L2[i][5], L2[i][6], L2[i][7],\
                [L1[i+2][3], L1[i+2][9], L1[i+2][21], L1[i+2][27], L1[i+2][33], L1[i+2][39]]))
    else:
        print("Encountered unknown nutrient type in append_to_list():\nL2[%d][1] = %s" % (i, L2[i][1]))

def read_data(carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list, daily_calorie):
    """
    Read in the user-specified calorie and store it as total_calorie
    Read in Nutrient.xlsx and Emission.xlsx
    Store each ingredient as an Ingredient object, assigning each property accordingly
    Append this object to carb_list, fat_list,... according to its nutrient type
    """
    #assign total calorie
    print("Please input the number of calories you want in a meal (kcal): ")
    try:
        total_calorie = float(input())
        if total_calorie > (daily_calorie/3)*1.2 or total_calorie < (daily_calorie/3)*0.8:
          print("Value is outside of the recommended range. Assigning recommended calories by default.")
          total_calorie = daily_calorie/3
    except ValueError:
        print("Error: The input is not a numeric value.\n Assigning recommended calories by default.")
        total_calorie = daily_calorie/3

    #read in dataset
    with open('Emission.csv', 'r') as fh:
        L1 = fh.readlines() #ingredient data starts from L1[3]
    with open('Nutrient.csv','r') as fh:
        L2 = fh.readlines() #ingredient data starts from L2[1]

    #store relevant data
    for i in range(1,len(L2)):
        L1[i+2] = L1[i+2].split(',')
        L2[i] = L2[i].split(',')
        for j in range(4, 8):
            L2[i][j] = (float) (L2[i][j])
        for j in range(6):
            L1[i+2][6*j+3] = (float) (L1[i+2][6*j+3])
        append_to_list(carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list, i, L1, L2)
    return total_calorie

## Functions that find and output: the "best" meal


These functions find the combination of ingredients that has the least environmental impact and has the specifed total calories

In [ ]:
import copy

def find_min_emmision(veg_weight, fruit_weight, CPF_ratio, meal_table, total_calorie, carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list, function):
    """
    Find the combination of ingredients with the lowest emission with the following loop

    for every combination of ingredients (one from each type):
        calculate the total calories of the vegetable, fruit and extra based on their weight
        assign the remaining calories to carb, protein and fruit according to CPF_ratio
        calculate the emmision caused by this meal

    Output the best combination in a 6*2 array.
    The first column contains the Ingredient variable
    The second column contains the
    """
    func_dict = {"Land Use": 0, "GHG Emissions": 1, "Acidifying Emissions": 2, "Eutrophying Emissions": 3, "Freshwater Withdrawals": 4, "Stress-Weighted Water Use": 5}
    min_emission = 999999.0
    for i in range(len(protein_list)):
        for j in range(len(carb_list)):
            for k in range(len(fat_list)):
                for l in range(len(fruit_list)):
                    for m in range(len(vegetable_list)):
                        for n in range(len(extra_list)):
                            #determine the weight of all ingredients
                            veg_cal = veg_weight*vegetable_list[m].calorie_per_unit
                            fruit_cal = fruit_weight*fruit_list[l].calorie_per_unit
                            extra_cal = extra_list[n].serving_portion*extra_list[n].calorie_per_unit
                            remaining_cal = total_calorie - veg_cal - fruit_cal - extra_cal
                            carb_cal = remaining_cal*CPF_ratio[0]
                            prot_cal = remaining_cal*CPF_ratio[1]
                            fat_cal = remaining_cal*CPF_ratio[2]
                            carb_weight = carb_cal/carb_list[j].calorie_per_unit
                            protein_weight = prot_cal/protein_list[i].calorie_per_unit
                            fat_weight = fat_cal/fat_list[k].calorie_per_unit

                            #calculate the GHG emission
                            emission = 0
                            weight_list = [carb_weight, protein_weight, fat_weight, fruit_weight, veg_weight, extra_list[n].serving_portion]
                            ingredient_list = [carb_list[j], protein_list[i], fat_list[k], fruit_list[l], vegetable_list[m], extra_list[n]]
                            for o in range(len(weight_list)):
                                try:
                                    emission += weight_list[o]*ingredient_list[o].emissions[func_dict[function]]
                                except:
                                    print(f'Weight_list[{o}]:', type(weight_list[o]))
                                    print(f'ingredient_list[{o}].emissions:', type(ingredient_list[o].emissions[func_dict[function]]))
                            if emission < min_emission:
                                min_emission = emission
                                for o in range(6):
                                    meal_table[o] = [copy.deepcopy(weight_list[o]), copy.deepcopy(ingredient_list[o].ingredient),\
                                                     copy.deepcopy(weight_list[o]*ingredient_list[o].calorie_per_unit),\
                                                     copy.deepcopy(weight_list[o]*ingredient_list[o].protein_per_unit),\
                                                     copy.deepcopy(weight_list[o]*ingredient_list[o].carb_per_unit),\
                                                     copy.deepcopy(weight_list[o]*ingredient_list[o].fat_per_unit)]
    return

def print_table(meal_table, function):
    sum_carb = 0
    sum_protein = 0
    sum_fat = 0
    sum_calorie = 0
    print('The meal minimizing %s is composed of:' % function)
    print('=======================================================================================================================================')
    for i in range(6):
        sum_calorie += meal_table[i][2]
        sum_protein += meal_table[i][3]
        sum_carb += meal_table[i][4]
        sum_fat += meal_table[i][5]
        print('{:>10.2f} g of {:>20s}, contributing {:>10.2f} kcal, {:>10.2f} g protein, {:>10.2f} g carb, {:>10.2f} g fat\n'\
                .format(1000*meal_table[i][0], meal_table[i][1], meal_table[i][2], meal_table[i][3],  meal_table[i][4], meal_table[i][5]))
    print('=======================================================================================================================================')
    print('Total: {:>54.2f} kcal, {:>10.2f} g of protein, {:>10.2f} g of carb, {:>10.2f} g of fat'.format(sum_calorie, sum_protein, sum_carb, sum_fat))
    return


In [ ]:
# @title Functions that read in additional user settings
def ingredient_requirement(ingredient_list, carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list):
    """
    This function reads in an ingredient that the user wants included in
    """
    list_array = [carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list]
    name_array = [[],[],[],[],[],[]]
    for i in range(len(name_array)):
        for j in range(len(list_array[i])):
            name_array[i].append(list_array[i][j].ingredient)
    while True:
        print("Please specify an ingredient.\n\
                Enter 'list' for the list of ingredients.\n\
                Enter 'apply' to apply settings.")
        nutrient_check = [0, 0, 0, 0, 0, 0]
        choice = input()
        if choice == "apply":
            for i in range(len(ingredient_list)):
                if ingredient_list[i] != 0:
                    for j in range(len(list_array[i])):
                        if list_array[i][j].ingredient == ingredient_list[i]:
                            ingredient = list_array[i][j]
                            break
                    list_array[i].clear()
                    list_array[i].append(ingredient)
            break
        elif choice == "list":
            for i in list_array:
                for j in i:
                    print("%s" % j.ingredient)
        elif choice in name_array[0]:
            if ingredient_list[0] == 0:
                print("Added %s to list" % choice)
            else:
                print("Replaced %s with %s" % (ingredient_list[0], choice))
            ingredient_list[0] = choice
        elif choice in name_array[1]:
            if ingredient_list[1] == 0:
                print("Added %s to list" % choice)
            else:
                print("Replaced %s with %s" % (ingredient_list[1], choice))
            ingredient_list[1] = choice
        elif choice in name_array[2]:
            if ingredient_list[2] == 0:
                print("Added %s to list" % choice)
            else:
                print("Replaced %s with %s" % (ingredient_list[2], choice))
            ingredient_list[2] = choice
        elif choice in name_array[3]:
            if ingredient_list[3] == 0:
                print("Added %s to list" % choice)
            else:
                print("Replaced %s with %s" % (ingredient_list[3], choice))
            ingredient_list[3] = choice
        elif choice in name_array[4]:
            if ingredient_list[4] == 0:
                print("Added %s to list" % choice)
            else:
                print("Replaced %s with %s" % (ingredient_list[4], choice))
            ingredient_list[4] = choice
        elif choice in name_array[5]:
            if ingredient_list[5] == 0:
                print("Added %s to list" % choice)
            else:
                print("Replaced %s with %s" % (ingredient_list[5], choice))
            ingredient_list[5] = choice
        else:
            print("Unknown input. Please input an ingredient on the list.")
    return

def impact_function_selection(impact_function_list):
    """
    This function prompts the user to select a way to evaluate environmental impact.
    Defaults to GHG emission if no input is received.
    """
    while True:
        print("Please input a measure of environmental impact.\n\
               Enter 'list' for the list of environmental impacts.\n\
               Enter 'apply' to apply settings.")
        choice = input()
        if choice == "apply":
            break
        elif choice == "list":
            print("List of environmental impacts:\n\
                    Land Use\n\
                    GHG Emissions\n\
                    Acidifying Emissions\n\
                    Eutrophying Emissions\n\
                    Freshwater Withdrawals")
        elif choice in ["Land Use","GHG Emissions","Eutrophying Emissions","Freshwater Withdrawals","Acidifying Emissions"]:
            if choice in impact_function_list:
                print("Already selected")
                continue
            else:
                print("Added %s to list" % choice)
                impact_function_list.append(choice)
        else:
            print("Unknown input. Please input an environmental impact on the list.")
    return

def optional_settings(impact_function_list, ingredient_list, carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list):
    """
    This function prompts the user to input additional settings, or go with the default options.
    """
    while True:
        print("Here are some additional settings you can specify:\n\
                1. Choose specific ingredient\n\
                2. Choose environmental impact evaluation method\n\
                3. Next step (applies default value for unspecified settings)\n\
                Enter the number of a setting:")
        try:
            choice = int(input())
        except ValueError as e:
            print("Input is not numeric. Please enter a number corresponding to an option.")
            continue
        if choice == 3:
            if impact_function_list == []:
                impact_function_list.append("GHG Emissions")
            break
        elif choice == 1:
            ingredient_requirement(ingredient_list, carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list)
        elif choice == 2:
            impact_function_selection(impact_function_list)
        else:
            print("Unknown input. Please enter a number corresponding to an option.")
    return


## Person class and associated functions to calculate the basal metabolic rate

In [ ]:
class Person :
    def __init__ (self) :
        self.sex = input("Please enter your biological sex (F or M): ")
        self.weight = int(input("Please enter your weight (in kg): "))
        self.height = int(input("Please enter your height (in cm): "))
        self.age = int(input("Please enter your age (between 0 and 150): "))

    def basalMetabolicRate (self) :
      """
      calculate the basal metabolic rate un kcal of a person
      Parameters in data mode: sex, weight, height, age
      Parameters in data/result mode : [none]
      Parameters in result mode : [none]
      Preconditions : sex is a str, weight, height, age are integers
      Postconditions : [none]
      Result: a floating-poiny number which is the result of the equation
      """
      metabolicRate = 0
      if (self.sex == "M") :
        metabolicRate = 10*self.weight + 6.25*self.height - 5*self.age + 5
      elif (self.sex == "F") :
        metabolicRate = 10*self.weight + 6.25*self.height - 5*self.age - 161
      else:
        print("Please enter your biological sex: F(female) or M(male)?")
      return metabolicRate

    def dailyEnergyRequirement (self,metabolicRate) :
        """
        calculate the daily energy requirement of a person
        Parameters in data mode: metabolicRate
        Parameters in data/result mode : [none]
        Parameters in result mode : [none]
        Preconditions : metabolicRate is a floating-point number, phyLevel is a str
        Postconditions : [none]
        Result: a floating-point number which is the result of the equation
        """
        phyLevel = input("Please enter your physical level (sedentary, light, moderate, intense, very intense): ")
        k = 0
        if (phyLevel == "sedentary") :
            k = 1.4
        elif (phyLevel == "light") :
            k = 1.6
        elif (phyLevel == "moderate") :
            k = 1.75
        elif (phyLevel == "intense") :
            k = 1.9
        elif (phyLevel == "very intense") :
            k = 2.1
        else:
            print("Please enter a valid keyword between our options! Thank you!")
        return k*metabolicRate

# Main Function

In [ ]:
#variable definition
total_calorie = 0 #to be assigned by read_data(), stored in kcal
veg_weight = 0.125 #the required kilograms of vegetables in each meal
fruit_weight = 0.050 #the required kilograms of fruits in each meal

carb_list = []
fat_list = []
protein_list = []
fruit_list = []
vegetable_list = []
extra_list = []

impact_function_list = []
ingredient_list = [0, 0, 0, 0, 0, 0]

meal_table = [0, 0, 0, 0, 0, 0]

CPF_ratio = (0.55, 0.20, 0.25)

#program starts here
person1 = Person()
print("Your basal metabolic rate is : ",person1.basalMetabolicRate())
daily_calorie = person1.dailyEnergyRequirement(person1.basalMetabolicRate())
print("The daily energy requirement for you is : {:.2f}".format(daily_calorie))
print("The recommended calorie is between {:.2f} and {:.2f}".format((daily_calorie/3)*0.8, (daily_calorie/3)*1.2))

"""
total_calorie = read_data(carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list)
find_min_emmision(veg_weight, fruit_weight, CPF_ratio, meal_table, total_calorie, carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list)
print_table(meal_table)
"""

total_calorie = read_data(carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list, daily_calorie)
optional_settings(impact_function_list, ingredient_list, carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list)
#print(impact_function_list)
for function in impact_function_list:
    find_min_emmision(veg_weight, fruit_weight, CPF_ratio, meal_table, total_calorie, carb_list, fat_list, protein_list, fruit_list, vegetable_list, extra_list, function)
    print_table(meal_table, function)


Please enter your biological sex (F or M): M
Please enter your weight (in kg): 68
Please enter your height (in cm): 175
Please enter your age (between 0 and 150): 22
Your basal metabolic rate is :  1668.75
Please enter your physical level (sedentary, light, moderate, intense, very intense): moderate
The daily energy requirement for you is : 2920.31
The recommended calorie is between 778.75 and 1168.12
Please input the number of calories you want in a meal (kcal): 
700
Value is outside of the recommended range. Assigning recommended calories by default.
Here are some additional settings you can specify:
                1. Choose specific ingredient
                2. Choose environmental impact evaluation method
                3. Next step (applies default value for unspecified settings)
                Enter the number of a setting:
1
Please specify an ingredient.
                Enter 'list' for the list of ingredients.
                Enter 'apply' to apply settings.
list
Wheat & Ry